In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import torch

In [ ]:
TRAIN_DATASET_PATH = "/content/drive/MyDrive/FYP/Davidson_Sampling/train_new.csv"
VALID_DATASET_PATH = "/content/drive/MyDrive/FYP/Davidson_Sampling/valid_new.csv"
TEST_DATASET_PATH = "/content/drive/MyDrive/FYP/Davidson_Sampling/test.csv"

In [ ]:
train_dataset= pd.read_csv(TRAIN_DATASET_PATH)

In [ ]:
train_dataset

In [ ]:
test_dataset = pd.read_csv(TEST_DATASET_PATH)

In [ ]:
valid_dataset = pd.read_csv(VALID_DATASET_PATH)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased")


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.to(device)
model.eval()

In [ ]:
seq_len = [len(tweet.split()) for tweet in train_dataset['tweet']]

In [ ]:
pd.Series(seq_len).hist(bins=40,color='firebrick')

In [ ]:
MAX_LENGTH = 20

In [ ]:
from torch.utils.data import DataLoader

#Train

In [ ]:
train_tokens= tokenizer.batch_encode_plus(train_dataset["tweet"].tolist(),padding='max_length',max_length=MAX_LENGTH,truncation=True)

In [ ]:
# import torch
# train_seq = torch.tensor(train_tokens['input_ids'])
# train_mask = torch.tensor(train_tokens['attention_mask'])
# train_y = torch.tensor(train_dataset['label'].tolist())

In [ ]:
# tokenized_train_datasets=train_tokens.remove_columns(['text'])

In [ ]:
# tokenized_train_datasets=t.with_format("torch")
from torch.utils.data import TensorDataset

# Assuming train_tokens contains the tokenized and encoded inputs
input_ids = torch.tensor(train_tokens['input_ids'])
attention_masks = torch.tensor(train_tokens['attention_mask'])

# Create a TensorDataset
train_new_dataset = TensorDataset(input_ids, attention_masks)

# Use the DataLoader with the TensorDataset
train_dataloader = DataLoader(train_new_dataset, batch_size=400, shuffle=False)

In [ ]:
train_dataset['label']

In [ ]:
train_dataloader

In [ ]:
# train_seq = torch.tensor(train_tokens['input_ids'])
# train_mask = torch.tensor(train_tokens['attention_mask'])
# train_y = torch.tensor(train_dataset['label'].tolist())

# test_seq = torch.tensor(test_tokens['input_ids'])
# test_mask = torch.tensor(test_tokens['attention_mask'])
# test_y = torch.tensor(test_dataset['label'].tolist())

In [ ]:
all_outputs_train_tensor = torch.tensor([], device=device)

In [ ]:
# for batch in train_dataloader:
#         print(batch)
#         batch = {k: v.to(device) for k, v in batch.items()}
#         with torch.no_grad():
#           outs = model(batch['input_ids'], token_type_ids=None, attention_mask=batch['attention_mask'])
#           bert_cls_hidden_state= outs[1]
#           print(bert_cls_hidden_state)
#           all_outputs_train_tensor = torch.cat([all_outputs_train_tensor, bert_cls_hidden_state], dim=0)


In [ ]:
for batch in train_dataloader:
    input_ids, attention_masks = batch
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    with torch.no_grad():
        outs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
        bert_cls_hidden_state = outs[1]
        all_outputs_train_tensor = torch.cat([all_outputs_train_tensor, bert_cls_hidden_state], dim=0)


In [ ]:
all_outputs_train_cpu = all_outputs_train_tensor.cpu()

In [ ]:
all_outputs_train_numpy = all_outputs_train_cpu.numpy()

In [ ]:
all_outputs_train_numpy

In [ ]:
import pandas as pd

# Assuming train_dataset['label'] is a numpy array
labels = train_dataset['label']

# Convert numpy arrays to pandas DataFrames
features_df = pd.DataFrame(all_outputs_train_numpy)
labels_df = pd.DataFrame(labels)

# Concatenate features and labels along columns axis
combined_df = pd.concat([features_df, labels_df], axis=1)




In [ ]:
combined_df

#Test

In [ ]:
test_tokens = tokenizer.batch_encode_plus(test_dataset['tweet'].tolist(),padding='max_length',max_length=MAX_LENGTH,truncation=True)

In [ ]:
# tokenized_train_datasets=t.with_format("torch")
from torch.utils.data import TensorDataset

# Assuming train_tokens contains the tokenized and encoded inputs
input_ids = torch.tensor(test_tokens['input_ids'])
attention_masks = torch.tensor(test_tokens['attention_mask'])

# Create a TensorDataset
test_new_dataset = TensorDataset(input_ids, attention_masks)

# Use the DataLoader with the TensorDataset
test_dataloader = DataLoader(test_new_dataset, batch_size=400, shuffle=False)

In [ ]:
all_outputs_test_tensor = torch.tensor([], device=device)

In [ ]:
# for batch in test_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         with torch.no_grad():
#           outs = model(batch['input_ids'], token_type_ids=None, attention_mask=batch['attention_mask'])
#           b_logit_pred = outs.logits

#           pred_label = torch.sigmoid(b_logit_pred)

#           all_outputs_test_tensor = torch.cat([all_outputs_test_tensor, pred_label], dim=0)
#           print(all_outputs_test_tensor)

In [ ]:
for batch in test_dataloader:
    input_ids, attention_masks = batch
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    with torch.no_grad():
        outs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
        bert_cls_hidden_state = outs[1]
        all_outputs_test_tensor = torch.cat([all_outputs_test_tensor, bert_cls_hidden_state], dim=0)


In [ ]:
all_outputs_test_cpu = all_outputs_test_tensor.cpu()

In [ ]:
all_outputs_test_numpy = all_outputs_test_cpu.numpy()

In [ ]:
import pandas as pd

# Assuming train_dataset['label'] is a numpy array
labels = test_dataset['label']

# Convert numpy arrays to pandas DataFrames
features_test_df = pd.DataFrame(all_outputs_test_numpy)
labels_test_df = pd.DataFrame(labels)

# Concatenate features and labels along columns axis
combined_test_df = pd.concat([features_test_df, labels_test_df], axis=1)




In [ ]:
combined_test_df

In [ ]:
# with torch.no_grad():
#     train_outputs = model(input_ids=train_seq, attention_mask=train_mask)

# # Extract the embeddings from the BERT model
# train_embeddings = train_outputs.last_hidden_state[:, 0, :].numpy()

In [ ]:
x_train_features = combined_df.iloc[:, :-1]
y_train_lables = combined_df['label']

In [ ]:
x_test_features = combined_test_df.iloc[:, :-1]
y_test_lables = combined_test_df['label']

#Valid

In [ ]:
valid_tokens = tokenizer.batch_encode_plus(valid_dataset['tweet'].tolist(),padding='max_length',max_length=MAX_LENGTH,truncation=True)

In [ ]:
# tokenized_train_datasets=t.with_format("torch")
from torch.utils.data import TensorDataset

# Assuming train_tokens contains the tokenized and encoded inputs
input_ids = torch.tensor(valid_tokens['input_ids'])
attention_masks = torch.tensor(valid_tokens['attention_mask'])

# Create a TensorDataset
valid_new_dataset = TensorDataset(input_ids, attention_masks)

# Use the DataLoader with the TensorDataset
valid_dataloader = DataLoader(valid_new_dataset, batch_size=400, shuffle=False)

In [ ]:
all_outputs_valid_tensor = torch.tensor([], device=device)

In [ ]:
for batch in valid_dataloader:
    input_ids, attention_masks = batch
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    with torch.no_grad():
        outs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
        bert_cls_hidden_state = outs[1]
        all_outputs_valid_tensor = torch.cat([all_outputs_valid_tensor, bert_cls_hidden_state], dim=0)


In [ ]:
all_outputs_valid_cpu = all_outputs_valid_tensor.cpu()

In [ ]:
all_outputs_valid_numpy = all_outputs_valid_cpu.numpy()

In [ ]:
import pandas as pd

# Assuming train_dataset['label'] is a numpy array
labels = valid_dataset['label']

# Convert numpy arrays to pandas DataFrames
features_valid_df = pd.DataFrame(all_outputs_valid_numpy)
labels_valid_df = pd.DataFrame(labels)

# Concatenate features and labels along columns axis
combined_valid_df = pd.concat([features_valid_df, labels_valid_df], axis=1)




In [ ]:
combined_valid_df

In [ ]:
x_valid_features = combined_valid_df.iloc[:, :-1]
y_valid_lables = combined_valid_df['label']

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
def score(y_test_lables,y_pred):
  accuracy = accuracy_score(y_test_lables, y_pred)
  precision = precision_score(y_test_lables, y_pred,average='macro')
  recall = recall_score(y_test_lables, y_pred,average='macro')
  f1 = f1_score(y_test_lables, y_pred,average='macro')
  conf_matrix = confusion_matrix(y_test_lables, y_pred)
  return accuracy,precision,recall,f1,conf_matrix

#LogisticRegression

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
# logistic_regression = LogisticRegression()
# param_grid = {
#     'C': [0.001, 0.01, 0.1],
#     'penalty': ['l1', 'l2'],
#     'solver': ['liblinear','saga']
# }

# grid_search = GridSearchCV(estimator=logistic_regression, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# grid_search.fit(x_valid_features, y_valid_lables)

# print("Best Parameters:", grid_search.best_params_)

In [ ]:
# y_pred = grid_search.predict(x_test_features)


In [ ]:
# accuracy,precision,recall,f1,conf_matrix = score (y_test_lables,y_pred)

In [ ]:
# print(f1)
# print(accuracy)
# print(recall)
# print(precision)
# print(conf_matrix)

#XGBoost

In [ ]:
# !pip install xgboost
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV

# # Define the hyperparameter grid
# param_grid = {
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.1, 0.01, 0.001],
#     'subsample': [0.5, 0.7, 1]
# }

# # Create the XGBoost model object
# xgb_model = XGBClassifier()

# # Create the GridSearchCV object
# grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')

# # Fit the GridSearchCV object to the training data
# grid_search.fit(x_valid_features, y_valid_lables)

# # Print the best set of hyperparameters and the corresponding score
# print("Best set of hyperparameters: ", grid_search.best_params_)

In [ ]:
# y_pred = grid_search.predict(x_test_features)

In [ ]:
# accuracy,precision,recall,f1,conf_matrix = score(y_test_lables,y_pred)

In [ ]:
# print(f1)
# print(accuracy)
# print(recall)
# print(precision)
# print(conf_matrix)

#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

# Define hyperparameters to tune
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(x_valid_features, y_valid_lables)

print("Best Parameters:", grid_search.best_params_)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best Parameters: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}


In [ ]:
y_pred = grid_search.predict(x_test_features)

In [ ]:
accuracy,precision,recall,f1,conf_matrix = score (y_test_lables,y_pred)

In [ ]:
print(f1)
print(accuracy)
print(recall)
print(precision)
print(conf_matrix)

#SVC

In [ ]:
#Hyper Parameter Tuning For SVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# defining parameter range
# param_grid = {'C': [0.1, 1, 10, 100, 1000],
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf']}
param_grid = {'C': [0.1, 1, 10],
             'gamma': [1, 0.1, 0.01],
             'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, cv=5, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(x_valid_features, y_valid_lables)
grid.best_params_


In [ ]:
y_pred = grid.predict(x_test_features)

In [ ]:
accuracy,precision,recall,f1,conf_matrix = score (y_test_lables,y_pred)

In [ ]:
print(f1)
print(accuracy)
print(recall)
print(precision)
print(conf_matrix)